<a href="https://colab.research.google.com/github/FabianSossa/PNL_UBA_TP/blob/main/Desafio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Desafio 2

### Consigna

Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.

Probar términos de interés y explicar similitudes en el espacio de embeddings.Intentar plantear y probar tests deanalogías.

Graficar los embeddingsresultantes.Sacar conclusiones.

### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará de la fuente Project Gutenberg (https://www.gutenberg.org/) El Toro : A motor car story of interior Cuba by E. Ralph Estep.


In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 680.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 18.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 re

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset un archivo en texto plano La Biblia en España, Tomo I (de 3) by George Borrow

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('https://www.gutenberg.org/cache/epub/51019/pg51019.txt', sep='/n', header=None)
df.head()

/tmp/ipython-input-2-1113889590.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('https://www.gutenberg.org/cache/epub/51019/pg51019.txt', sep='/n', header=None)


,0
0,The Project Gutenberg eBook of La Biblia en Es...
1,This ebook is for the use of anyone anywhere i...
2,most other parts of the world at no cost and w...
3,"whatsoever. You may copy it, give it away or r..."
4,of the Project Gutenberg License included with...


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 7351


### 1 - Preprocesamiento

In [ ]:
# from keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK or spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:20]

[['the',
  'project',
  'gutenberg',
  'ebook',
  'of',
  'la',
  'biblia',
  'en',
  'españa',
  'tomo',
  'i',
  'de',
  '3'],
 ['this',
  'ebook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'in',
  'the',
  'united',
  'states',
  'and'],
 ['most',
  'other',
  'parts',
  'of',
  'the',
  'world',
  'at',
  'no',
  'cost',
  'and',
  'with',
  'almost',
  'no',
  'restrictions'],
 ['whatsoever',
  'you',
  'may',
  'copy',
  'it',
  'give',
  'it',
  'away',
  'or',
  're',
  'use',
  'it',
  'under',
  'the',
  'terms'],
 ['of',
  'the',
  'project',
  'gutenberg',
  'license',
  'included',
  'with',
  'this',
  'ebook',
  'or',
  'online'],
 ['at',
  'www',
  'gutenberg',
  'org',
  'if',
  'you',
  'are',
  'not',
  'located',
  'in',
  'the',
  'united',
  'states'],
 ['you',
  'will',
  'have',
  'to',
  'check',
  'the',
  'laws',
  'of',
  'the',
  'country',
  'where',
  'you',
  'are',
  'located'],
 ['before', 'using', 'this', 'ebook'],
 ['titl

### 2 - Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
def train_word2vec_model(sentences, sg, window):
    model = Word2Vec(
        sentences=sentences,  # Corpus tokenizado
        vector_size=300,      # Dimensionalidad de los vectores
        window=window,        # Ventana (2 o 4)
        min_count=5,          # Frecuencia mínima para incluir una palabra
        negative=20,          # Cantidad de negative samples
        workers=1,            # Cantidad de núcleos a utilizar
        sg=sg                 # 0: CBOW, 1: Skipgram
    )
    model.build_vocab(sentences)  # Construir el vocabulario
    model.train(sentences, total_examples=model.corpus_count, epochs=10)  # Entrenar el modelo
    return model

In [ ]:
# Entrenar 4 modelos con las configuraciones solicitadas
models = {}
for sg in [0, 1]:  # 0: CBOW, 1: Skipgram
    for window in [2, 4]:  # Ventanas de 2 y 4 palabras
        key = f"sg={sg}_window={window}"
        models[key] = train_word2vec_model(sentence_tokens, sg=sg, window=window)
        print(f"Modelo {key}:")
        print("Cantidad de docs en el corpus:", models[key].corpus_count)
        print("Cantidad de words distintas en el corpus:", len(models[key].wv.index_to_key))
        print("---")

Modelo sg=0_window=2:
Cantidad de docs en el corpus: 7351
Cantidad de words distintas en el corpus: 1942
---


Modelo sg=0_window=4:
Cantidad de docs en el corpus: 7351
Cantidad de words distintas en el corpus: 1942
---


Modelo sg=1_window=2:
Cantidad de docs en el corpus: 7351
Cantidad de words distintas en el corpus: 1942
---


Modelo sg=1_window=4:
Cantidad de docs en el corpus: 7351
Cantidad de words distintas en el corpus: 1942
---


### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
# Entrenar los modelos con el callback definido
def train_models_with_callback(models, sentence_tokens, epochs=20):
    for key, model in models.items():
        print(f"Entrenando modelo {key}...")
        model.train(
            sentence_tokens,
            total_examples=model.corpus_count,
            epochs=epochs,
            compute_loss=True,
            callbacks=[callback()]  # Utilizamos tu callback definido
        )
        print(f"Entrenamiento del modelo {key} finalizado.\n")

# Entrenar los 4 modelos con tu callback
train_models_with_callback(models, sentence_tokens, epochs=20)

Entrenando modelo sg=0_window=2...
Loss after epoch 0: 163238.34375
Loss after epoch 1: 160980.40625
Loss after epoch 2: 159446.90625
Loss after epoch 3: 160032.78125
Loss after epoch 4: 158205.1875
Loss after epoch 5: 155467.0625
Loss after epoch 6: 146156.1875
Loss after epoch 7: 133712.75
Loss after epoch 8: 132307.25
Loss after epoch 9: 130048.125
Loss after epoch 10: 129407.875
Loss after epoch 11: 128396.75
Loss after epoch 12: 126548.25
Loss after epoch 13: 125863.25
Loss after epoch 14: 121123.125
Loss after epoch 15: 111356.25
Loss after epoch 16: 110585.25
Loss after epoch 17: 109926.5
Loss after epoch 18: 109516.75


Loss after epoch 19: 109280.0
Entrenamiento del modelo sg=0_window=2 finalizado.

Entrenando modelo sg=0_window=4...
Loss after epoch 0: 162368.25
Loss after epoch 1: 160681.0625
Loss after epoch 2: 159900.03125
Loss after epoch 3: 160951.90625
Loss after epoch 4: 159733.9375
Loss after epoch 5: 157450.5
Loss after epoch 6: 148134.1875
Loss after epoch 7: 136603.75
Loss after epoch 8: 135294.125
Loss after epoch 9: 133365.75
Loss after epoch 10: 132636.625
Loss after epoch 11: 131973.25
Loss after epoch 12: 130136.0
Loss after epoch 13: 129484.5
Loss after epoch 14: 120837.125
Loss after epoch 15: 113726.25
Loss after epoch 16: 113243.25
Loss after epoch 17: 112458.0
Loss after epoch 18: 112076.0


Loss after epoch 19: 111761.25
Entrenamiento del modelo sg=0_window=4 finalizado.

Entrenando modelo sg=1_window=2...
Loss after epoch 0: 349463.5625
Loss after epoch 1: 344847.0625
Loss after epoch 2: 343252.375
Loss after epoch 3: 304371.25
Loss after epoch 4: 298842.625
Loss after epoch 5: 294990.5
Loss after epoch 6: 277611.625
Loss after epoch 7: 259048.25
Loss after epoch 8: 257411.25
Loss after epoch 9: 253675.75
Loss after epoch 10: 253065.25
Loss after epoch 11: 251893.5
Loss after epoch 12: 248740.0
Loss after epoch 13: 247294.5
Loss after epoch 14: 243399.0
Loss after epoch 15: 231352.0
Loss after epoch 16: 229267.0
Loss after epoch 17: 227420.0
Loss after epoch 18: 227254.0


Loss after epoch 19: 225458.5
Entrenamiento del modelo sg=1_window=2 finalizado.

Entrenando modelo sg=1_window=4...
Loss after epoch 0: 488545.15625
Loss after epoch 1: 482085.28125
Loss after epoch 2: 446108.3125
Loss after epoch 3: 431534.5
Loss after epoch 4: 408053.25
Loss after epoch 5: 377476.25
Loss after epoch 6: 375287.5
Loss after epoch 7: 373435.5
Loss after epoch 8: 371534.0
Loss after epoch 9: 365560.75
Loss after epoch 10: 345317.0
Loss after epoch 11: 339592.0
Loss after epoch 12: 334824.5
Loss after epoch 13: 332419.5
Loss after epoch 14: 331020.5
Loss after epoch 15: 332503.5
Loss after epoch 16: 330063.0
Loss after epoch 17: 326483.5
Loss after epoch 18: 327920.0
Loss after epoch 19: 324601.5
Entrenamiento del modelo sg=1_window=4 finalizado.



### 4 - Ensayar

In [ ]:
# Función para obtener las palabras más similares a una palabra objetivo en los 4 modelos
def get_most_similar_for_all_models(models, positive=None, negative=None, topn=10):
    for key, model in models.items():
        try:
            print(f"Modelo {key} - Palabras más similares (positive={positive}, negative={negative}):")
            similar_words = model.wv.most_similar(positive=positive, negative=negative, topn=topn)
            for similar_word, similarity in similar_words:
                print(f"  {similar_word}: {similarity:.4f}")
            print("---")
        except KeyError as e:
            print(f"Modelo {key} - Error: {str(e)}. Una de las palabras no está en el vocabulario.\n")

In [ ]:

# Ejemplo: Obtener palabras más relacionadas con "dios" y opcionalmente con negativas
get_most_similar_for_all_models(models, positive=["dios"], negative=None, topn=10)

Modelo sg=0_window=2 - Palabras más similares (positive=['dios'], negative=None):
  maneras: 0.8721
  escribir: 0.8701
  sobra: 0.8697
  servir: 0.8586
  baltasar: 0.8566
  justicia: 0.8552
  raza: 0.8534
  aun: 0.8509
  míos: 0.8477
  lenguaje: 0.8446
---
Modelo sg=0_window=4 - Palabras más similares (positive=['dios'], negative=None):
  servir: 0.8796
  escribir: 0.8770
  sobra: 0.8747
  viajes: 0.8742
  justicia: 0.8681
  ideas: 0.8679
  enemigos: 0.8677
  ladrones: 0.8660
  baltasar: 0.8652
  casos: 0.8615
---
Modelo sg=1_window=2 - Palabras más similares (positive=['dios'], negative=None):
  maneras: 0.7812
  pregunté—: 0.7651
  raza: 0.7606
  escribir: 0.7551
  servir: 0.7543
  sean: 0.7493
  darle: 0.7450
  ministros: 0.7429
  continuamente: 0.7399
  entender: 0.7316
---
Modelo sg=1_window=4 - Palabras más similares (positive=['dios'], negative=None):
  justicia: 0.7079
  pregunté—: 0.6974
  palos: 0.6881
  campos: 0.6853
  servir: 0.6736
  amor: 0.6725
  propia: 0.6718
  entend

In [ ]:
# Ejemplo: Obtener palabras más relacionadas con "dios" y opcionalmente con negativas
get_most_similar_for_all_models(models, positive=None, negative=["dios"], topn=10)

Modelo sg=0_window=2 - Palabras más similares (positive=None, negative=['dios']):
  —la: 0.0452
  —el: -0.0189
  —un: -0.0491
  hacia: -0.0685
  donde: -0.0715
  del: -0.0787
  desde: -0.1432
  hasta: -0.1516
  el: -0.1525
  hora: -0.1773
---
Modelo sg=0_window=4 - Palabras más similares (positive=None, negative=['dios']):
  —la: 0.1312
  hacia: 0.0471
  donde: 0.0459
  —un: 0.0322
  —el: -0.0015
  del: -0.0040
  desde: -0.0121
  hasta: -0.0321
  llegamos: -0.0425
  hora: -0.0513
---
Modelo sg=1_window=2 - Palabras más similares (positive=None, negative=['dios']):
  donde: -0.1337
  —un: -0.1460
  hacia: -0.1816
  llegamos: -0.1926
  hasta: -0.1957
  desde: -0.2019
  —la: -0.2025
  cerro: -0.2295
  aldea: -0.2322
  —el: -0.2370
---
Modelo sg=1_window=4 - Palabras más similares (positive=None, negative=['dios']):
  donde: -0.0950
  desde: -0.1091
  hasta: -0.1123
  —un: -0.1146
  media: -0.1196
  cerro: -0.1396
  llegamos: -0.1556
  torre: -0.1579
  noche: -0.1646
  aldea: -0.1647
---


### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import plotly.express as px

# Reducir las dimensiones de los vectores de un modelo Word2Vec
def reduce_dimensions(model, num_dimensions=2):
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    reduced_vectors = tsne.fit_transform(vectors)
    return reduced_vectors, labels

# Graficar los embeddings para todos los modelos
def plot_embeddings_for_models(models, max_words=200):
    for key, model in models.items():
        print(f"Generando visualización para el modelo {key}...")
        vecs, labels = reduce_dimensions(model)

        # Limitar el número de palabras a graficar
        fig = px.scatter(
            x=vecs[:max_words, 0],
            y=vecs[:max_words, 1],
            text=labels[:max_words],
            title=f"Visualización de Embeddings: {key}"
        )

        fig.show(renderer="colab")

# Graficar los embeddings de los 4 modelos entrenados
plot_embeddings_for_models(models, max_words=150)

Generando visualización para el modelo sg=0_window=2...


Generando visualización para el modelo sg=0_window=4...


Generando visualización para el modelo sg=1_window=2...


Generando visualización para el modelo sg=1_window=4...


In [18]:
# Graficar los embeddings en 3D para el primer modelo entrenado
MAX_WORDS = 200  # Número máximo de palabras para graficar

# Reducir las dimensiones a 3D para el primer modelo
first_model_key = list(models.keys())[0]  # Obtén el primer modelo de la lista
first_model = models[first_model_key]  # Modelo seleccionado
vecs, labels = reduce_dimensions(first_model, num_dimensions=3)


fig = px.scatter_3d(
    x=vecs[:MAX_WORDS, 0],
    y=vecs[:MAX_WORDS, 1],
    z=vecs[:MAX_WORDS, 2],
    text=labels[:MAX_WORDS],
    title=f"Visualización de Embeddings en 3D: {first_model_key}"
)

# Ajustar el tamaño de los marcadores
fig.update_traces(marker_size=2)

# Mostrar el gráfico (ajusta el renderer según tu entorno)
fig.show(renderer="colab")

### Tests de Analogía

In [20]:
# Función para realizar un Test de Analogía con el primer modelo
def analogy_test_first_model(models, positive_words, negative_words=None, topn=1):
    """
    Realiza un Test de Analogía utilizando el primer modelo de Word2Vec.

    :param models: Diccionario de modelos entrenados.
    :param positive_words: Lista de palabras que suman significado (e.g., ["rey", "mujer"]).
    :param negative_words: Lista de palabras que restan significado (e.g., ["hombre"]).
    :param topn: Número de resultados a devolver.
    """
    # Seleccionar el primer modelo del diccionario
    first_model_key = list(models.keys())[0]
    model = models[first_model_key]

    try:
        # Realizar la consulta de analogía
        results = model.wv.most_similar(positive=positive_words, negative=negative_words, topn=topn)
        print(f"Analogía ({positive_words} - {negative_words}):")
        print(f"Resultados del modelo {first_model_key}:")
        for word, similarity in results:
            print(f"  {word}: {similarity:.4f}")
    except KeyError as e:
        print(f"Error: Una de las palabras no está en el vocabulario - {e}")

# Dios es a amor como hombre es a pecado.
analogy_test_first_model(models, positive_words=["justicia", "amor"], negative_words=["pecado"], topn=5)

# Rey es a sabio como hombre es a justo.
analogy_test_first_model(models, positive_words=["rey", "sabio"], negative_words=["hombre"], topn=5)
# Luz es a tinieblas como verdad es a mentira
analogy_test_first_model(models, positive_words=["luz", "verdad"], negative_words=["tinieblas"], topn=5)
#Padre es a hijo como madre es a hija.
analogy_test_first_model(models, positive_words=["padre", "hijo"], negative_words=["madre"], topn=5)

Error: Una de las palabras no está en el vocabulario - "Key 'pecado' not present in vocabulary"
Error: Una de las palabras no está en el vocabulario - "Key 'sabio' not present in vocabulary"
Analogía (['luz', 'verdad'] - ['tinieblas']):
Resultados del modelo sg=0_window=2:
  cuadra: 0.7134
  gitana: 0.7041
  razón: 0.6980
  burra: 0.6732
  guerra: 0.6468
Analogía (['padre', 'hijo'] - ['madre']):
Resultados del modelo sg=0_window=2:
  compañero: 0.8749
  guía: 0.8100
  ro: 0.8081
  criado: 0.7917
  espíritu: 0.7854
